# 1 climatezone per country / state

In [ ]:
import pandas as pd
import src.utils.USClimateAndWeatherHelper as WaH
import src.utils.preProcessingHelper as preProcessingHelper
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## US States

- decide on a states climatezone based on the counties climatezones and weighted by population. In other words: in what climate zones do people in that state live We do this to anaylze in what conditions beer-reviewers live.
County climatezone data from US department for energy: https://www.energy.gov/sites/prod/files/2015/10/f27/ba_climate_region_guide_7.3.pdf
- county population data from: https://www.census.gov/data/tables/time-series/demo/popest/2020s-counties-total.html
- used formatted table from github: https://gist.github.com/philngo/d3e251040569dba67942#file-climate_zones-csv
- state abbreviation list:https://www.faa.gov/air_traffic/publications/atpubs/cnt_html/appendix_a.html


In [2]:
# Define paths
countiesClimate_df_path = "data/climatezones/climate_zones.csv"
countiesPopulation_df_path = "data/climatezones/countyPopulation.xlsx"

# Get the climate data with population
climate_data_with_population = WaH.add_population_to_climate_data(countiesClimate_df_path, countiesPopulation_df_path)

# Get the most populated climate zone for each state
most_populated_climate_zone = WaH.get_most_populated_climate_zone(climate_data_with_population)

# View the result
most_populated_climate_zone.head()


/home/nfilimonov/Documents/GitHub/ada-2024-project-leonardodatavinci/src/utils/USClimateAndWeatherHelper.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  match = countiesPopulation_df[countiesPopulation_df['County'].str.contains(row['County'], case=False, na=False)]


,State,IECC Climate Zone,Population
0,AK,7,555076.0
3,AL,3,4377717.0
4,AR,3,3483542.0
6,AS,1,0.0
7,AZ,2,6116085.0


# 2 Extracting weather data from excel files

In [3]:
precipitation_df = WaH.extract_data_from_excel('data/weather-data/cru-x0.5_timeseries_pr_timeseries_monthly_1901-2022_mean_historical_cru_ts4.07_mean.xlsx')
temperature_df = WaH.extract_data_from_excel('data/weather-data/cru-x0.5_timeseries_tas_timeseries_monthly_1901-2022_mean_historical_cru_ts4.07_mean.xlsx')

#export to csv
precipitation_df.to_csv('data/weather-data/us-precipitations.csv', index=False)
temperature_df.to_csv('data/weather-data/us-temperatures.csv', index=False)

precipitation_df

,code,state,value,year,month
0,USA,United States of America,46.10,1901,1
1,USA.2593214,Alabama,116.85,1901,1
2,USA.2593215,Alaska,35.22,1901,1
3,USA.2593216,Arizona,24.17,1901,1
4,USA.2593217,Arkansas,60.62,1901,1
...,...,...,...,...,...
76123,USA.2593260,Virginia,96.01,2022,12
76124,USA.2593261,Washington,169.15,2022,12
76125,USA.2593262,West Virginia,82.01,2022,12
76126,USA.2593263,Wisconsin,50.55,2022,12


In [4]:
# Compute oldest and newest year
oldest_year = min(precipitation_df['year'].min(), temperature_df['year'].min())
newest_year = max(precipitation_df['year'].max(), temperature_df['year'].max())

# Print the oldest and newest year
print(f"Oldest year: {oldest_year}")
print(f"Newest year: {newest_year}")


Oldest year: 1901
Newest year: 2022


In [5]:
month = 1
year = 1986
state = "United States of America"

precipitation_USA_1986_1 = WaH.value_from_date_and_state(precipitation_df, year, month, state)
print(f"Precipitation in {state} in {month}-{year}: {precipitation_USA_1986_1}")

Precipitation in United States of America in 1-1986: 36.83


# For TIM /!\ here, replace path_to_Tim_data by actual dataset

In [6]:
path_to_Tim_data = "/home/nfilimonov/Documents/GitHub/beerAdvocateRatings.csv"

beer_data = pd.read_csv(path_to_Tim_data)

In [7]:
beer_data

,rating,country_name,country_code3,state,date_object,month,year
0,2.88,United States,USA,Washington,2015-08-20 12:00:00,8,2015
1,3.67,United States,USA,New York,2009-02-20 12:00:00,2,2009
2,3.73,United Kingdom,GBR,NaN,2006-03-13 12:00:00,3,2006
3,3.98,United Kingdom,GBR,NaN,2004-12-01 12:00:00,12,2004
4,4.00,United Kingdom,GBR,NaN,2004-08-30 12:00:00,8,2004
...,...,...,...,...,...,...,...
8393027,3.80,United States,USA,Connecticut,2006-02-27 12:00:00,2,2006
8393028,2.80,United States,USA,Maryland,2006-02-19 12:00:00,2,2006
8393029,1.54,United States,USA,Virginia,2004-08-10 12:00:00,8,2004
8393030,3.50,NaN,NaN,NaN,2012-07-08 12:00:00,7,2012


In [8]:
# Filter for US only
beer_data_with_weather = beer_data[beer_data["country_name"] == "United States"]

#Filter for years between 2002 and 2017
beer_data_with_weather = beer_data_with_weather[(beer_data_with_weather["year"] >= 2002) & (beer_data_with_weather["year"] <= 2017)]

# print shape
print("original shape: ", beer_data.shape)
print("new shape: ", beer_data_with_weather.shape)

# beer_data_with_weather = beer_data_with_weather.head(1000)

beer_data_with_weather = WaH.apply_value_from_date_and_state(beer_data_with_weather,precipitation_df, "Precipitation")
beer_data_with_weather = WaH.apply_value_from_date_and_state(beer_data_with_weather,temperature_df, "Temperature")
beer_data_with_weather = preProcessingHelper.merge_with_states(beer_data_with_weather)


beer_data_with_weather

original shape:  (8393032, 7)
new shape:  (7301788, 7)


,rating,country_name,country_code3,state,date_object,month,year,Precipitation,Temperature,abbreviation
0,2.88,United States,USA,Washington,2015-08-20 12:00:00,8,2015,40.63,19.08,WA
1,3.67,United States,USA,New York,2009-02-20 12:00:00,2,2009,43.11,-4.40,NY
2,3.25,United States,USA,Kansas,2014-12-01 12:00:00,12,2014,26.37,1.84,KS
3,3.50,United States,USA,New Jersey,2014-05-29 12:00:00,5,2014,91.57,17.21,NJ
4,3.50,United States,USA,North Carolina,2014-02-26 12:00:00,2,2014,91.12,6.16,NC
...,...,...,...,...,...,...,...,...,...,...
7301783,3.18,United States,USA,Rhode Island,2004-11-15 12:00:00,11,2004,108.62,6.62,RI
7301784,3.80,United States,USA,Connecticut,2006-02-27 12:00:00,2,2006,54.93,-1.24,CT
7301785,2.80,United States,USA,Maryland,2006-02-19 12:00:00,2,2006,41.60,2.06,MD
7301786,1.54,United States,USA,Virginia,2004-08-10 12:00:00,8,2004,121.49,22.24,VA


# 3 Data Analysis

In [ ]:
# Create a 'date' column for easier grouping
beer_data_with_weather['date'] = pd.to_datetime(beer_data_with_weather[['year', 'month']].assign(day=1))

# 1. Plot month/year vs. number of rows using Plotly

# Count the number of rows per unique date
row_counts = beer_data_with_weather['date'].value_counts().sort_index()

# Create a line plot using Plotly Express
fig = px.line(
    x=row_counts.index,
    y=row_counts.values,
    labels={'x': 'Date', 'y': 'Number of Rows'},
    title='Number of Rows per Month/Year'
)

# Update layout for better appearance
fig.update_layout(
    xaxis=dict(
        tickformat='%Y-%m',
        tickangle=45
    ),
    xaxis_tickformatstops=[
        dict(dtickrange=[None, None], value="%Y-%m")
    ]
)

#Save the plot
fig.write_html("illustrations/Nicolas/number_of_rows_per_month_year.html")

# 2. Plot a timeline with start month/year and end month/year using Plotly

# Get the start and end dates
start_date = beer_data_with_weather['date'].min()
end_date = beer_data_with_weather['date'].max()

# Create a figure
fig_timeline = go.Figure()

# Add a line and markers to represent the timeline
fig_timeline.add_trace(go.Scatter(
    x=[start_date, end_date],
    y=[0, 0],
    mode='lines+markers',
    marker=dict(size=10),
    line=dict(width=2),
    showlegend=False
))

# Update layout for the timeline
fig_timeline.update_layout(
    title=f'Timeline of Dataset<br><sup>Start: {start_date.strftime("%Y-%m")} | End: {end_date.strftime("%Y-%m")}</sup>',
    xaxis_title='Date',
    yaxis=dict(
        showticklabels=False,
        showgrid=False,
        zeroline=False
    ),
    xaxis=dict(
        tickformat='%Y-%m',
        tickangle=45
    ),
    height=200,
    margin=dict(l=50, r=50, t=80, b=50)
)


#Save the plot
fig_timeline.write_html("illustrations/Nicolas/timeline.html")

In [ ]:
# Average temperature per month for each state
avg_temp_state = beer_data_with_weather.groupby(['state', 'month'])['Temperature'].mean().reset_index()

# Plot
fig = px.line(avg_temp_state, x='month', y='Temperature', color='state', 
              title="Average Monthly Temperature by State",
              labels={'Temperature': 'Avg Temperature (°C)', 'month': 'Month', 'state': 'State'})
fig.update_layout(xaxis=dict(tickmode='array', tickvals=list(range(1, 13)), ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']))

#save
fig.write_html("illustrations/Nicolas/avg_temp_state.html")


In [11]:
# Calculate average temperature by state and month
temp_heatmap = beer_data_with_weather.groupby(['state', 'month'])['Temperature'].mean().reset_index()

# Create heatmap with inverted axes (months on y-axis, states on x-axis)
fig = px.imshow(temp_heatmap.pivot(index='month', columns='state', values='Temperature'),
                labels=dict(x="State", y="Month", color="Avg Temperature (°C)"),
                y=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                title="Average Temperature by Month and State")


# Save plot
fig.write_html("illustrations/Nicolas/heatmap.html")

In [12]:
# Calculate the average temperature by state abbreviation
state_avg_temp = beer_data_with_weather.groupby('abbreviation')['Temperature'].mean().reset_index()

state_avg_temp = preProcessingHelper.merge_with_abbreviations(state_avg_temp)

state_avg_temp.head()

,abbreviation,Temperature,state
0,AK,-3.243337,Alaska
1,AL,18.054488,Alabama
2,AR,16.759027,Arkansas
3,AZ,16.623173,Arizona
4,CA,14.999503,California


In [ ]:
# Create the choropleth map
fig = px.choropleth(
    state_avg_temp,
    locations='abbreviation',  # Column with state abbreviations
    locationmode='USA-states',  # Specifies that locations are USA states
    color='Temperature',  # Data to color the states by
    scope='usa',  # Focuses the map on the USA
    color_continuous_scale='Viridis',  # Color scale
    labels={'Temperature': 'Avg Temperature (°C)','state':'State'},
    title='Average Temperature by State in the USA',
    hover_data={'state': True}           # Adds state name to hover information
)

# Save 
fig.write_html("illustrations/Nicolas/temperature.html")


In [14]:
# Calculate the average temperature by state abbreviation
state_monthly_avg_temp = beer_data_with_weather.groupby(['abbreviation', 'month'])['Temperature'].mean().reset_index()

state_monthly_avg_temp = preProcessingHelper.merge_with_abbreviations(state_monthly_avg_temp)

state_monthly_avg_temp.head()

,abbreviation,month,Temperature,state
0,AK,1,-16.599246,Alaska
1,AK,2,-14.565560,Alaska
2,AK,3,-12.961200,Alaska
3,AK,4,-3.625553,Alaska
4,AK,5,4.294199,Alaska


In [ ]:
# Define fixed color scale range
min_temp = state_monthly_avg_temp['Temperature'].min()  # e.g., -35
max_temp = state_monthly_avg_temp['Temperature'].max()  # e.g., 30

# Create the timelapse choropleth map
fig = px.choropleth(
    state_monthly_avg_temp,
    locations='abbreviation',           # Column with state abbreviations
    locationmode='USA-states',           # Specifies that locations are USA states
    color='Temperature',                 # Data to color the states by
    hover_name='state',                  # Display full state name on hover
    scope='usa',                         # Focuses the map on the USA
    color_continuous_scale='Viridis',    # Color scale
    labels={'Temperature': 'Avg Temperature (°C)', 'month': 'Month'},
    title='Average Temperature by State in the USA Over Time',
    animation_frame='month',             # Create animation based on the month
    range_color=[min_temp, max_temp]     # Set fixed scale for the color range
)

# Customize animation speed
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 500  # Adjust speed (milliseconds per frame)
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 300

# Save Timelapse
fig.write_html("illustrations/Nicolas/timelapse_temperature.html")


# 4 Archive - World Climate Zones

In [16]:
from climateZoneLoaderWorldwide import generate_climate_map, get_climate_class, generate_station_map, load_station_data

In [17]:
# Define period and file path
period = '1986-2010'
raster_file = f'data/climatezones/Map_KG-Global/KG_{period}.grd'

In [18]:
# Example usage:
generate_climate_map(raster_file, 10)
print(get_climate_class(16.375, 48.210, raster_file))  # Example: Vienna, Austria

Using downsample factor: 10
Non-empty points identified: 93312
Processing point 0 of 93312...
Processing point 10000 of 93312...
Processing point 20000 of 93312...
Processing point 30000 of 93312...
Processing point 40000 of 93312...
Processing point 50000 of 93312...
Processing point 60000 of 93312...
Processing point 70000 of 93312...
Processing point 80000 of 93312...
Processing point 90000 of 93312...
Cfb


In [19]:
# Load the data and generate the map
df = load_station_data('data/climatezones/weather_stations_world.txt')
generate_station_map(df)

In [ ]:
# Define the column specifications: (start position, end position)
colspecs = [
    (0, 7),    # Wmo#
    (8, 9),    # R
    (10, 15),   # Lat
    (16, 22),  # Lon
    (23, 47),  # Country Name
    (48, 73),  # Station Name
    (74, 78),  # Statn Elev (m)
    (79, 85),  # Barom Elev (.1m)
    (86, 96),  # Local Statn #
    (97, 99),  # Separator
    (100, 105),  # Mean Stn Pres
    (106, 110), # Mean Slvl Pres
    (111, 115),# Mean Temp
    (116, 120),# Dly Totl Prec
    (121, 125),# Mean Max Temp
    (126, 130),# Mean Min Temp
    (131, 135),# Mean RH
    (136, 141) # Total # Obs
]

# Define the column names
column_names = [
    'Wmo#', 'R', 'Lat', 'Lon', 'Country Name', 'Station Name',
    'Statn Elev (m)', 'Barom Elev (.1m)', 'Local Statn #', 'Separator',
    'Mean Stn Pres', 'Mean Slvl Pres', 'Mean Dly Temp', 'Totl Prec',
    'Mean Max Temp', 'Mean Min Temp', 'Mean RH', 'Total # Obs'
]

# Read the data using read_fwf
df = pd.read_fwf(
    'data/climatezones/weather_stations_world.txt',
    colspecs=colspecs,
    skiprows=4,  # Skip header rows
    names=column_names,
    encoding='ISO-8859-1'  # Specify encoding to handle special characters
)

# Display the first few rows of the DataFrame
print(df.head(6))


   Wmo#  R    Lat     Lon Country Name       Station Name  Statn Elev (m)  \
0  1001  1  7056N  00840W       NORWAY          JAN MAYEN              10   
1  2013  1  6744N  01728E       SWEDEN           RITSEM A             532   
2  2019  1  6825N  01810E       SWEDEN       KATTERJåKK A             516   
3  2020  1  6825N  01810E       SWEDEN         KATTERJåKK             515   
4  2024  1  6730N  01818E       SWEDEN  STORA SJöFALLET A             425   
5  2029  1  6755N  01836E       SWEDEN          TARFALA A            1150   

   Barom Elev (.1m)  Local Statn # Separator  Mean Stn Pres  Mean Slvl Pres  \
0            -99999            NaN         :              0               0   
1              5330            NaN         :             34              68   
2              5180            NaN         :             34              70   
3              5173            NaN         :              0               0   
4              4270            NaN         :              0      